In [1]:
import os
import sys
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import pickle

import multiprocessing
num_cpus = multiprocessing.cpu_count()

# Setup
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(num_cpus, device)


8 cuda:0


In [13]:
# parameters
dim_embedding = 4096
k = 30

#----------------------------------------------------------------------
# functions
def default_image_loader(path):
    return Image.open(path).convert('RGB')

class TripletImageLoader(torch.utils.data.Dataset):
    def __init__(self, base_path, triplets_file_name, transform=None,
                 loader=default_image_loader):
        self.base_path = base_path  
        self.filenamelist = []
        triplets = []
        for line in open(triplets_file_name):
            triplets.append((line.split()[0], line.split()[1], line.split()[2])) # Q, P, N
        self.triplets = triplets
        self.transform = transform
        self.loader = loader

    def __getitem__(self, index):
        path1, path2, path3 = self.triplets[index]
        img1 = self.loader(os.path.join(self.base_path,path1))
        img2 = self.loader(os.path.join(self.base_path,path2))
        img3 = self.loader(os.path.join(self.base_path,path3))
        if self.transform is not None:
            img1 = self.transform(img1)
            img2 = self.transform(img2)
            img3 = self.transform(img3)

        return img1, img2, img3

    def __len__(self):
        return len(self.triplets)
    
class EmbeddingImageLoader(torch.utils.data.Dataset):
    def __init__(self, base_path, image_file_name, transform=None,
                 loader=default_image_loader):
        self.base_path = base_path  
        self.filenamelist = []
        files = []
        for line in open(image_file_name):
            files.append((line.rstrip('\n')))
        self.files = files
        self.transform = transform
        self.loader = loader

    def __getitem__(self, index):
        path = self.files[index]
        img = self.loader(os.path.join(self.base_path,path))
        if self.transform is not None:
            img = self.transform(img)

        return img

    def __len__(self):
        return len(self.files)



In [14]:
# transformations
pretrain_image_size = 224

transform_train = transforms.Compose(
    [torchvision.transforms.Resize(pretrain_image_size),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
transform_test = transforms.Compose(
    [torchvision.transforms.Resize(pretrain_image_size),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

#----------------------------------------------------------------------
# data

base_path = 'tiny-imagenet-200'
train_sampler_file = "train_sampler.txt"
trainset = TripletImageLoader(base_path, train_sampler_file, transform = transform_train)
trainloader = DataLoader(trainset, batch_size= 10, shuffle=False, num_workers=10)

db_file = 'db.txt'
val_file = 'val.txt'

db_classes = np.array(pickle.load(open('db_classes.pkl','rb')))
val_classes = np.array(pickle.load(open('val_classes.pkl','rb')))

dataset = EmbeddingImageLoader(base_path, db_file, transform = transform_test)
valset = EmbeddingImageLoader(base_path, val_file, transform = transform_test)

testloader_db = DataLoader(dataset, batch_size=100, shuffle=False, num_workers=4)
testloader_val = DataLoader(valset, batch_size=100, shuffle=False, num_workers=4)



In [15]:
# def save_checkpoint(state, epoch, filename='checkpoint_res101.pth.tar'):
#     torch.save(state, filename)
#     if is_best:
#         torch.save(state, 'model_best_res101.pth.tar')

# network
class TripletNet(nn.Module):
    def __init__(self, embedding_net):
        super(TripletNet, self).__init__()
        self.embedding_net = embedding_net

    def forward(self, Q, P, N):
        Q_embedding = self.embedding_net(Q)
        P_embedding = self.embedding_net(P)
        N_embedding = self.embedding_net(N)
        return Q_embedding, P_embedding, N_embedding

resnet = models.resnet50(pretrained=True)

for p in resnet.parameters():
    p.requires_grad = False

# change output layer
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, dim_embedding)



model = TripletNet(resnet)

model.to(device)



TripletNet(
  (embedding_net): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (downsample): Sequential(
          (

In [16]:
start_epoch = 0
best_prec1 = 0.0
# if len(sys.argv) == 2:
#     cp_file = sys.argv[1]
#     cp = torch.load(cp_file)
#     start_epoch = cp['epoch']
#     best_prec1 = cp['best_prec1']
#     model.load_state_dict(cp['state_dict'])
#     print('cp loaded')

criterion = nn.TripletMarginLoss(margin = 1)
# optimizer = optim.SGD(net.parameters(), lr=0.001)
optimizer = optim.SGD(resnet.fc.parameters(), lr=0.001,momentum=0.9)

                                                                # 0.001  0.0001 0.00001
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40000,50000,60000,70000,80000], gamma=0.9)
#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size, gamma=0.)
# Train
val_percision_over_time = []
best_acc = max(0.0,best_prec1)

model.train()
for epoch in range(1):  # loop over the dataset multiple times
    #scheduler.step()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        model.train()
        # get the inputs
        
        scheduler.step()
        Qin, Pin, Nin = data
        Qin, Pin, Nin = Qin.to(device), Pin.to(device), Nin.to(device)
        optimizer.zero_grad()
        # forward
        eQ, eP, eN = model(Qin, Pin, Nin)
        
        loss_tripletNet = criterion(eQ, eP, eN)
        loss_embeddingNet = eQ.norm(2) + eP.norm(2) + eN.norm(2)
        #loss = loss_tripletNet + 0.001 * loss_embeddingNet
        loss = loss_tripletNet
        loss.backward()
        for group in optimizer.param_groups:
            for p in group['params']:
                state = optimizer.state[p]
                if('step' in state and state['step']>=1024):
                    state['step'] = 1000

        optimizer.step()
        
	# print running_loss once in a while
        loss_t = 100
        running_loss += loss.item()
        if (i+1) % loss_t == 0:
            print('average',loss_t,'epoch loss at batch number',(i+1),':\t',running_loss/loss_t)
            running_loss = 0.0
            
        if (i+1) % 10000 == 0:
            
            filename = 'Epoch_' + str((i+1)/10000) +'_res50_4096_new.pth.tar' 
            torch.save(model.state_dict(), filename)

print('Finished Training')


BrokenPipeError: [Errno 32] Broken pipe

In [5]:
str((9999+1)/10000)
(9999+1)%10000

0